In [3]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
washroom = '/content/drive/MyDrive/AIML/Xebia Internship/data/357washroom.csv'

In [6]:
import pandas as pd

# Load GTFS files into DataFrames
washroom_df = pd.read_csv(washroom)

In [7]:
print("Agency Data:")
print(washroom_df.head())

Agency Data:
   Sl.No.           Topic   Old Title  \
0       1  Public Toilets  CTU‐03‐001   
1       2  Public Toilets  CTU‐03‐002   
2       3  Public Toilets  CTU‐03‐003   
3       4  Public Toilets  CTU‐03‐004   
4       5  Public Toilets  CTU‐03‐005   

                                             Address   Latitute  Longitude  
0  CTU at old R K Ashram Marg towards Ashirwad Ol...  28.629837  77.201384  
1  CTU in JJ Camp C‐33, G‐Point  Jhuggi, Kali Bar...  28.627700  77.202090  
2         CTU in JJ Camp C‐31, Kali Bari Marg No.‐ 1  28.629580  77.202190  
3  CTU in JJ Camp C‐31Back side, Kali Bari Marg N...  28.630220  77.202820  
4  CTU in JJ Camp C‐31Back side, Kali Bari Marg N...  28.630600  77.203620  


In [8]:
washroom_df.isnull().sum()

Sl.No.       0
Topic        0
Old Title    0
Address      0
Latitute     1
Longitude    1
dtype: int64

In [9]:
washroom_df =washroom_df.dropna()

In [11]:
latitude, longitude = washroom_df['Latitute'].median(), washroom_df['Longitude'].median()

In [12]:
import folium

In [15]:
map1 = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, label,code in zip(washroom_df['Latitute'], washroom_df['Longitude'], washroom_df['Address'],washroom_df['Old Title']):
    folium.CircleMarker(
        [lat,long],
        radius = 1,
        color = 'Blue',
        popup = (label,code),
        fill=True,
        fill_color='Red',
        fill_opacity=0.5
    ).add_to(map1)
map1

In [20]:
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def extract_keywords(query):
    doc = nlp(query)
    keywords = [token.text.lower() for token in doc if token.ent_type_ in {'GPE', 'LOC', 'FAC'} or token.pos_ in {'NOUN', 'PROPN'}]
    return keywords

def find_washroom(query):
    keywords = extract_keywords(query)
    matches = washroom_df[washroom_df['Address'].str.lower().apply(lambda x: any(keyword in x.lower() for keyword in keywords))]
    if not matches.empty:
        return matches.to_dict(orient='records')
    return "Sorry, I couldn't find nearby washroom."

In [21]:
# Test the system with user queries
queries = [
    " washroom at Princess Park Part‐I?",
    "At Anand Vihar",
    "near safdarjung enclave "
]

for query in queries:
    result = find_washroom(query)
    print(f"Query: {query}\nAnswer: {result}\n")

Query:  washroom at Princess Park Part‐I?
Answer: [{'Sl.No.': 12, 'Topic': 'Public Toilets', 'Old Title': 'CTU‐04‐005', 'Address': 'CTU at Park Lane Jhuggi', 'Latitute': 28.62532, 'Longitude': 77.19656}, {'Sl.No.': 15, 'Topic': 'Public Toilets', 'Old Title': 'CTU‐05‐003', 'Address': 'CTU at Princess Park Part‐I', 'Latitute': 28.61743, 'Longitude': 77.23296}, {'Sl.No.': 16, 'Topic': 'Public Toilets', 'Old Title': 'CTU‐05‐004', 'Address': 'CTU at Princess Park Part‐II', 'Latitute': 28.61795, 'Longitude': 77.23307}, {'Sl.No.': 58, 'Topic': 'Public Toilets', 'Old Title': 'PTU(Petrol Pump 22)', 'Address': 'Humayun Rd  Pandara Flats  Sujan Singh Park  New Delhi  Delhi 110003  India', 'Latitute': 28.6021667, 'Longitude': 77.2298455}, {'Sl.No.': 83, 'Topic': 'Public Toilets', 'Old Title': 'PTU‐01‐008', 'Address': 'PTU at Parking Area Super Bazar(Outer circus CP )', 'Latitute': 28.6333191, 'Longitude': 77.2231495}, {'Sl.No.': 94, 'Topic': 'Public Toilets', 'Old Title': 'PTU‐01‐019', 'Address': 

In [22]:
# Test the system with user queries
queries = [
    "Jasola.", "INA Market"
]

for query in queries:
    result = find_washroom(query)
    print(f"Query: {query}\nAnswer: {result}\n")


Query: Jasola.
Answer: Sorry, I couldn't find nearby washroom.

Query: INA Market
Answer: [{'Sl.No.': 8, 'Topic': 'Public Toilets', 'Old Title': 'CTU‐04‐001', 'Address': 'CTU near Chelmford Club JJ Camp Raisina Road No.1', 'Latitute': 28.61789, 'Longitude': 77.21422}, {'Sl.No.': 9, 'Topic': 'Public Toilets', 'Old Title': 'CTU‐04‐002', 'Address': 'CTU near Chelmsford Club JJ Camp Raisina RoadNo. II', 'Latitute': 28.6178858, 'Longitude': 77.2143976}, {'Sl.No.': 43, 'Topic': 'Public Toilets', 'Old Title': 'PTU(Petrol Pump 07)', 'Address': 'D‐II 262  Vinay Marg  D1 Flats  Block D 2  Chanakyapuri  New Delhi  Delhi 110021  India', 'Latitute': 28.5907657, 'Longitude': 77.1959754}, {'Sl.No.': 56, 'Topic': 'Public Toilets', 'Old Title': 'PTU(Petrol Pump 20)', 'Address': '40  Sri Aurobindo Marg  Aviation Colony  INA Colony  New Delhi  Delhi 110023  India', 'Latitute': 28.573952, 'Longitude': 77.208872}, {'Sl.No.': 60, 'Topic': 'Public Toilets', 'Old Title': 'PTU(Petrol Pump 24)', 'Address': '1/1